In [1]:
import numpy as np
import networkx as nx
from sklearn.decomposition import TruncatedSVD as TSVD
#from graspy.embed import AdjacencySpectralEmbed

from tqdm import tqdm_notebook as tqdm

In [2]:
def gaussian_covariance(X, Y, bandwidth = 0.5):
    diffs = np.expand_dims(X, 1) - np.expand_dims(Y, 0)
    return np.exp(-0.5 * np.sum(diffs**2, axis=2) / bandwidth**2)

In [3]:
def statistic(X, Y):
    N, _ = X.shape
    M, _ = Y.shape
    x_stat = np.sum(gaussian_covariance(X, X, 0.5) - np.eye(N))/(N*(N-1))
    y_stat = np.sum(gaussian_covariance(Y, Y, 0.5) - np.eye(M))/(M*(M-1))
    xy_stat = np.sum(gaussian_covariance(X, Y, 0.5))/(N*M)
    return x_stat - 2*xy_stat + x_stat

In [4]:
def ASE(A):
    tsvd = TSVD()
    vecs, vals = tsvd.fit(A).components_, tsvd.singular_values_
    vecs_2 = np.array([vecs[0, :], vecs[1, :]])
    if vecs_2[0,0] < 0:
        vecs_2 *= -1  
    X_hat = vecs_2.T @ np.diag(vals[:2]**(1/2))
    return X_hat

def ASE_graspy(A): #too SLOW! Takes like 100 hrs... can't validate using this.
    ase = AdjacencySpectralEmbed(algorithm='randomized')
    X_hat = ase.fit_transform(A)
    return X_hat

In [5]:
def bootstrap(X, Y, M, alpha = 0.05):
    N, _ = X.shape
    M, _ = Y.shape
    
    statistics = np.zeros(M)
    for i in range(M):
        bs_X = X[np.random.choice(np.arange(0,N), size = int(N/2), replace = False)]
        bs_Y = Y[np.random.choice(np.arange(0,M), size = int(M/2), replace = False)]
        statistics[i] = statistic(bs_X, bs_Y)
        
    sorted_ = np.sort(statistics)
    rej_ind = int(np.ceil(((1 - alpha)*M)))
    return sorted_[rej_ind]

In [6]:
def gen_data(n, eps):
    pi = [0.4, 0.6]
    sizes = [int(pi[0]*n), int(pi[1]*n)]

    probsA = np.array([
        [0.5, 0.2],
        [0.2, 0.5]])
    
    probsB = np.array([
        [0.5 + eps, 0.2],
        [0.2, 0.5 + eps]])
    
    G1 = nx.stochastic_block_model(sizes, probsA)
    A1 = nx.to_numpy_array(G1)

    G2 = nx.stochastic_block_model(sizes, probsA)
    A2 = nx.to_numpy_array(G2)
    return sizes, probsA, probsB, A1, A2

In [7]:
def estimated_power(n, eps, M, alpha, iters):
    sizes, probsA, probsB, A1, A2 = gen_data(n, eps)
    
    X1_hat = ASE(A1)
    X2_hat = ASE(A2)
    critical_value = bootstrap(X1_hat, X2_hat, M, alpha)
    
    rejections = 0
    for i in range(iters):
        G3 = nx.stochastic_block_model(sizes, probsA)
        A = nx.to_numpy_array(G3)
        G4 = nx.stochastic_block_model(sizes, probsB)
        B = nx.to_numpy_array(G4)
        X_hat = ASE(A)
        Y_hat = ASE(B)

        U = statistic(X_hat, Y_hat)
        if U > critical_value:
            rejections += 1
    return rejections/iters

In [12]:
def monte_carlo(ns, eps, M = 200, alpha = 0.05, iters = 500):
    powers = np.zeros(shape = (len(ns),len(eps)))
    for i in tqdm(range(len(ns))):
        for j in range(len(eps)):
            powers[i,j] = np.array(estimated_power(ns[i], eps[j], M, alpha, iters))
    return powers

In [13]:
monte_ns = [100, 200, 500, 1000]
monte_eps = [0.0, 0.02, 0.05, 0.1]
power_table = monte_carlo(ns = monte_ns, eps = monte_eps)

In [14]:
print("Paper results")
print(np.array([[.06,.09,.27],[.09,.17,.83],[.1,.43,1],[.14,1,1]]))
print("Sim results")
print(power_table)

Paper results
[[0.06 0.09 0.27]
 [0.09 0.17 0.83]
 [0.1  0.43 1.  ]
 [0.14 1.   1.  ]]
Sim results
[[0.    0.    0.01  0.204]
 [0.    0.    0.254 1.   ]
 [0.    0.002 1.    1.   ]
 [0.    0.134 1.    1.   ]]
